In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [3]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
sub = pd.read_csv('../input/sample_submission.csv')

In [4]:
print(sub.head())

   enrollee_id  target
0        16548       0
1        12036       0
2        11061       0
3         5032       0
4        17599       0


In [5]:
print(train.shape)
train.sample(5)

(18359, 14)


,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
12259,20131,city_103,0.92,Male,Has relevent experience,no_enrollment,Graduate,Business Degree,2,NaN,NaN,1,8,0
12388,28004,city_103,0.92,NaN,Has relevent experience,no_enrollment,Graduate,STEM,9,100-500,Pvt Ltd,2,75,0
14807,1335,city_103,0.92,NaN,Has relevent experience,no_enrollment,Graduate,STEM,6,10000+,Pvt Ltd,4,17,0
4053,31890,city_103,0.92,Male,No relevent experience,Part time course,Graduate,STEM,4,50-99,Pvt Ltd,1,28,0
17393,24487,city_103,0.92,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,>4,50,0


In [5]:
print(test.shape)
test.sample(5)

(15021, 13)


,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
4094,22959,city_21,0.624,Male,Has relevent experience,no_enrollment,Graduate,STEM,<1,50-99,Pvt Ltd,1,102
1469,27181,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,6,10000+,Pvt Ltd,1,78
4722,7102,city_40,0.776,Male,Has relevent experience,no_enrollment,NaN,NaN,14,10/49,NGO,4,48
9815,9362,city_103,0.920,Female,Has relevent experience,no_enrollment,Graduate,STEM,2,10000+,Pvt Ltd,1,61
38,5931,city_50,0.896,Male,Has relevent experience,no_enrollment,Masters,STEM,5,1000-4999,Pvt Ltd,1,124


- Check for nans

In [6]:
train.isnull().sum()/train.shape[0]

enrollee_id               0.000000
city                      0.000000
city_development_index    0.000000
gender                    0.223215
relevent_experience       0.000000
enrolled_university       0.018628
education_level           0.024892
major_discipline          0.154584
experience                0.003214
company_size              0.260308
company_type              0.274470
last_new_job              0.019990
training_hours            0.000000
target                    0.000000
dtype: float64

In [7]:
for col in train.columns:
    print(f'{col}-> {train[col].nunique()} : {train[col].dtype} -> {train[col].isnull().sum()}')
    print('\n')

enrollee_id-> 18359 : int64 -> 0


city-> 123 : object -> 0


city_development_index-> 93 : float64 -> 0


gender-> 3 : object -> 4098


relevent_experience-> 2 : object -> 0


enrolled_university-> 3 : object -> 342


education_level-> 5 : object -> 457


major_discipline-> 6 : object -> 2838


experience-> 22 : object -> 59


company_size-> 8 : object -> 4779


company_type-> 6 : object -> 5039


last_new_job-> 6 : object -> 367


training_hours-> 241 : int64 -> 0


target-> 2 : int64 -> 0




In [8]:
cols_to_impute = []
for col in train.columns:
    if train[col].isnull().sum() > 0:
        print(f'{col} -> {train[col].isnull().sum()}')
        cols_to_impute.append(col)

gender -> 4098
enrolled_university -> 342
education_level -> 457
major_discipline -> 2838
experience -> 59
company_size -> 4779
company_type -> 5039
last_new_job -> 367


In [9]:
for col in train.columns:
    if train[col].dtype == 'object' and train[col].nunique() > 2:
        print(f'{col} -> {train[col].nunique()}')

city -> 123
gender -> 3
enrolled_university -> 3
education_level -> 5
major_discipline -> 6
experience -> 22
company_size -> 8
company_type -> 6
last_new_job -> 6


In [10]:
for col in train.columns:
    if train[col].dtype in [float, int]:
        print(f'{col} -> {train[col].nunique()}')

enrollee_id -> 18359
city_development_index -> 93
training_hours -> 241
target -> 2


In [11]:
for col in train.columns:
    if train[col].dtype in [float, int]:
        print(train[col].describe())

count    18359.000000
mean     16729.360096
std       9643.749725
min          1.000000
25%       8378.500000
50%      16706.000000
75%      25148.500000
max      33380.000000
Name: enrollee_id, dtype: float64
count    18359.000000
mean         0.847140
std          0.110189
min          0.448000
25%          0.796000
50%          0.910000
75%          0.920000
max          0.949000
Name: city_development_index, dtype: float64
count    18359.000000
mean        65.899014
std         60.885300
min          1.000000
25%         23.000000
50%         47.000000
75%         89.000000
max        336.000000
Name: training_hours, dtype: float64
count    18359.000000
mean         0.132088
std          0.338595
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: target, dtype: float64


In [12]:
train['type'] = 'train'
test['type'] = 'test'

## Preprocess draft

- handle missing values

In [13]:
tt_len = test.shape[0]
all_data = pd.concat([train, test])
print(all_data.shape)
all_data.tail()

(33380, 15)


,city,city_development_index,company_size,company_type,education_level,enrolled_university,enrollee_id,experience,gender,last_new_job,major_discipline,relevent_experience,target,training_hours,type
15016,city_46,0.762,500-999,Pvt Ltd,Masters,no_enrollment,11308,>20,NaN,>4,STEM,Has relevent experience,NaN,68,test
15017,city_21,0.624,1000-4999,Pvt Ltd,Masters,Full time course,14612,4,Male,1,STEM,Has relevent experience,NaN,320,test
15018,city_16,0.910,1000-4999,Pvt Ltd,High School,no_enrollment,33346,9,Male,4,NaN,Has relevent experience,NaN,13,test
15019,city_64,0.666,NaN,NaN,Graduate,Full time course,14506,5,NaN,1,STEM,No relevent experience,NaN,38,test
15020,city_21,0.624,NaN,NaN,Graduate,no_enrollment,32641,3,Male,NaN,NaN,No relevent experience,NaN,100,test


In [14]:
for col in cols_to_impute:
    all_data[col] = all_data[col].fillna(all_data[col].value_counts().index[0]) 

In [15]:
all_data.isnull().sum()

city                          0
city_development_index        0
company_size                  0
company_type                  0
education_level               0
enrolled_university           0
enrollee_id                   0
experience                    0
gender                        0
last_new_job                  0
major_discipline              0
relevent_experience           0
target                    15021
training_hours                0
type                          0
dtype: int64

In [16]:
def label_encode(data, feats):
    df = data.copy(deep = True)
    dict_of_dicts = {}
    for col in feats:
        length = []
        values = []
        encoder_dict = {}
        for idx, val in zip(range(len(df[col].unique())),df[col].unique()):
            if str(val) == 'nan':
                pass
            else:
                length.append(idx)
                values.append(val)
        for l,v in zip(length, values):
            encoder_dict[v] = l

        dict_of_dicts[col] = encoder_dict

        df[col] = df[col].replace(dict_of_dicts.get(col))
        
    return df

def CountEncoding(df, cols, df_test=None):
    for col in cols:
        frequencies = df[col].value_counts().reset_index()
        df_values = df[[col]].merge(frequencies, how='left', left_on=col, right_on='index').iloc[:,-1].values
        df[col+'_counts'] = df_values
        df_test_values = df_test[[col]].merge(frequencies, how='left', left_on=col, right_on='index').fillna(1).iloc[:,-1].values
        df_test[col+'_counts'] = df_test_values
    count_cols = [col+'_counts' for col in cols]
    return df, df_test

def binary_processor(data):
    df = data.copy()
            
    #label encoder for binary feats
    df = label_encode(df, feats = ['relevent_experience'])

    #sanity_check
    print('missing values for binary cols')
    for col in ['relevent_experience']:
        print(f'{col} null values: {df[col].isnull().sum()}/{df.shape[0]} - \
    prop: {round(df[col].isnull().sum()/df.shape[0], 3)}')
    
    return df

def full_nominal_processor(df):
    df = df.copy()
    nominal = []
    for col in train.columns:
        if train[col].dtype == 'object' and train[col].nunique() > 2:
            nominal.append(col)
            
    df = label_encode(df, feats = nominal)

    tr = df[df['type'] == 'train']
    tt = df[df['type'] == 'train']

    tr, tt = CountEncoding(tr, nominal, df_test = tt)

    df = pd.concat([tr,tt])


    return df

def other_preprocessing(df):
    df = df.copy()
    df['enrollee_id'] = df['enrollee_id'].astype('category')
    df['city_development_index'] = df['city_development_index'].astype('category')
    df['city_development_index'] = label_encode(df, feats = ['city_development_index'])
    df['training_hours'] = df['training_hours'].astype('category')
    df['training_hours'] = label_encode(df, feats = ['training_hours'])
    df['target'] = df['target'].astype('category')
    
    return df

In [17]:
#binary
all_data = binary_processor(all_data)
all_data = full_nominal_processor(all_data)
all_data = other_preprocessing(all_data)
all_data.head()

missing values for binary cols
relevent_experience null values: 0/33380 -     prop: 0.0


,city,city_development_index,company_size,company_type,education_level,enrolled_university,enrollee_id,experience,gender,last_new_job,...,type,city_counts,gender_counts,enrolled_university_counts,education_level_counts,major_discipline_counts,experience_counts,company_size_counts,company_type_counts,last_new_job_counts
0,0,0,0,0,0,0,23798,0,0,0,...,train,105,16982,14001,11226,16576,1159,2698,15090,7934
1,1,1,1,1,0,0,29166,1,0,0,...,train,142,16982,14001,11226,16576,602,1360,1038,7934
2,2,2,2,2,0,0,46,2,0,1,...,train,1654,16982,14001,11226,16576,1125,7899,996,2835
3,3,3,2,0,0,0,18527,1,0,0,...,train,119,16982,14001,11226,16576,602,7899,15090,7934
4,4,4,2,0,1,0,21751,3,0,1,...,train,271,16982,14001,4319,16576,755,7899,15090,2835


In [18]:
all_data.shape

(36718, 24)

In [19]:
train.shape

(18359, 15)

In [ ]:
df['company_size'] = label_encode(df, feats = ['company_size'])
df['company_type'] = df['company_type'].astype('category')
df['training_hours'] = label_encode(df, feats = ['training_hours'])
df['target'] = df['target'].astype('category')

In [21]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36718 entries, 0 to 18358
Data columns (total 24 columns):
city                          36718 non-null int64
city_development_index        36718 non-null object
company_size                  36718 non-null int64
company_type                  36718 non-null int64
education_level               36718 non-null int64
enrolled_university           36718 non-null int64
enrollee_id                   36718 non-null category
experience                    36718 non-null int64
gender                        36718 non-null int64
last_new_job                  36718 non-null int64
major_discipline              36718 non-null int64
relevent_experience           36718 non-null int64
target                        36718 non-null category
training_hours                36718 non-null object
type                          36718 non-null object
city_counts                   36718 non-null int64
gender_counts                 36718 non-null int64
enrolled_univer

In [25]:
train_non_cat = []
for col in all_data:
    if 'counts' in col:
        train_non_cat.append(col)

np.array(train_non_cat)

array(['city_counts', 'gender_counts', 'enrolled_university_counts',
       'education_level_counts', 'major_discipline_counts',
       'experience_counts', 'company_size_counts', 'company_type_counts',
       'last_new_job_counts'], dtype='<U26')

In [26]:
all_data.enrollee_id.nunique()

18359